In [1]:
# Respuesta
import os
os.environ['PYSPARK_PYTHON'] = '/usr/local/bin/python3.6'

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()



# Valores Ausentes

Los valores ausentes en _pyspark_ están identificados como _null_. El método `isNull` permite idenficar los registros nulos y `isNotNull` los no nulos.

In [2]:
from pyspark.sql import functions as F

In [3]:
vancouver_df = spark.read.csv('Data/crime_in_vancouver.csv', sep=',', header=True, inferSchema=True)

In [4]:
vancouver_df.filter(F.col('NEIGHBOURHOOD').isNull()).show(4)

+--------------------+----+-----+---+----+------+--------------------+-------------+---+---+
|                TYPE|YEAR|MONTH|DAY|HOUR|MINUTE|       HUNDRED_BLOCK|NEIGHBOURHOOD|  X|  Y|
+--------------------+----+-----+---+----+------+--------------------+-------------+---+---+
|Offence Against a...|2003|    5| 17|null|  null|OFFSET TO PROTECT...|         null|0.0|0.0|
|Offence Against a...|2003|    6|  3|null|  null|OFFSET TO PROTECT...|         null|0.0|0.0|
|Offence Against a...|2003|    7| 17|null|  null|OFFSET TO PROTECT...|         null|0.0|0.0|
|Offence Against a...|2003|    6| 26|null|  null|OFFSET TO PROTECT...|         null|0.0|0.0|
+--------------------+----+-----+---+----+------+--------------------+-------------+---+---+
only showing top 4 rows



In [5]:
vancouver_df.filter(F.col('NEIGHBOURHOOD').isNotNull()).show(4)

+--------------------+----+-----+---+----+------+--------------------+--------------------+---------+----------+
|                TYPE|YEAR|MONTH|DAY|HOUR|MINUTE|       HUNDRED_BLOCK|       NEIGHBOURHOOD|        X|         Y|
+--------------------+----+-----+---+----+------+--------------------+--------------------+---------+----------+
|  Theft from Vehicle|2003|    1|  7|  10|     0|CARDERO ST / W GE...|            West End|490503.48|5459766.67|
|  Theft from Vehicle|2003|    7| 27|   5|     0|    53XX CHAMBERS ST| Renfrew-Collingwood|496145.89|5453740.68|
|Break and Enter R...|2003|    3|  8|   4|    15|     19XX E 12TH AVE|Kensington-Cedar ...|495302.97|5456313.79|
|  Theft from Vehicle|2003|   10|  9|  16|     0|     16XX CHARLES ST|  Grandview-Woodland|494877.89| 5457816.4|
+--------------------+----+-----+---+----+------+--------------------+--------------------+---------+----------+
only showing top 4 rows



 

## Conteo de valores nulos

In [6]:
vancouver_df.filter(F.col('NEIGHBOURHOOD').isNull()).count()

58423

In [7]:
vancouver_df.filter(F.col('TYPE').isNull()).count()

0



### Porcentaje de ausentes por columna

El primer método es menos eficiente que el segundo ya que requiere ejecutar una acción por cada columna. Como norma general en Spark hay que intentar realizar el número mínimo de acciones.

In [8]:
n_rows_vancouver = vancouver_df.count()



__Método 1:__

In [9]:
%%time

for col in vancouver_df.columns:
    
    n_missing = vancouver_df.filter(F.col(col).isNull()).count()
    perc_missing = 100 * n_missing / n_rows_vancouver
    
    print(col, round(perc_missing, 2))

TYPE 0.0
YEAR 0.0
MONTH 0.0
DAY 0.0
HOUR 10.16
MINUTE 10.16
HUNDRED_BLOCK 0.0
NEIGHBOURHOOD 10.58
X 0.0
Y 0.0
CPU times: user 30.3 ms, sys: 9.04 ms, total: 39.3 ms
Wall time: 9.58 s




__Método 2:__

Para una única columna

In [10]:
vancouver_df.select(F.round(F.sum(F.col('NEIGHBOURHOOD').isNull().cast('int')) * 100 / n_rows_vancouver, 2)\
                      .alias('NEIGHBOURHOOD')).show()

+-------------+
|NEIGHBOURHOOD|
+-------------+
|        10.58|
+-------------+





Todas las columnas

In [11]:
%%time 

missing_ops = [F.round(F.sum(F.col(c).isNull().cast('int')) * 100 / n_rows_vancouver, 2).alias(c) 
               for c in vancouver_df.columns]

vancouver_df.select(missing_ops).show()

+----+----+-----+---+-----+------+-------------+-------------+---+---+
|TYPE|YEAR|MONTH|DAY| HOUR|MINUTE|HUNDRED_BLOCK|NEIGHBOURHOOD|  X|  Y|
+----+----+-----+---+-----+------+-------------+-------------+---+---+
| 0.0| 0.0|  0.0|0.0|10.16| 10.16|          0.0|        10.58|0.0|0.0|
+----+----+-----+---+-----+------+-------------+-------------+---+---+

CPU times: user 33.9 ms, sys: 7.76 ms, total: 41.7 ms
Wall time: 2.27 s


 

## Eliminación registros nulos

El método `dropNa` se utiliza para eliminar registros nulos. Con el parámetro `subset` se indican sobre qué columnas buscar nulos y el parámetro `how` selecciona con qué condición se elimina un registro.

In [12]:
vancouver_df.dropna(how='all').count()

552055

In [13]:
vancouver_df.dropna(how='any').count()

493619

In [14]:
vancouver_no_missing_df = vancouver_df.dropna(subset=['HOUR', 'MINUTE'])

In [15]:
vancouver_no_missing_df.select(missing_ops).show()

+----+----+-----+---+----+------+-------------+-------------+---+---+
|TYPE|YEAR|MONTH|DAY|HOUR|MINUTE|HUNDRED_BLOCK|NEIGHBOURHOOD|  X|  Y|
+----+----+-----+---+----+------+-------------+-------------+---+---+
| 0.0| 0.0|  0.0|0.0| 0.0|   0.0|          0.0|         0.42|0.0|0.0|
+----+----+-----+---+----+------+-------------+-------------+---+---+





## Imputación de valores nulos

`fillna` imputa los valores nulos de las columnas a un valor fijo elegido.

In [16]:
vancouver_df.show(3)

+--------------------+----+-----+---+----+------+--------------------+-------------------+---------+----------+
|                TYPE|YEAR|MONTH|DAY|HOUR|MINUTE|       HUNDRED_BLOCK|      NEIGHBOURHOOD|        X|         Y|
+--------------------+----+-----+---+----+------+--------------------+-------------------+---------+----------+
|Offence Against a...|2003|    5| 17|null|  null|OFFSET TO PROTECT...|               null|      0.0|       0.0|
|  Theft from Vehicle|2003|    1|  7|  10|     0|CARDERO ST / W GE...|           West End|490503.48|5459766.67|
|  Theft from Vehicle|2003|    7| 27|   5|     0|    53XX CHAMBERS ST|Renfrew-Collingwood|496145.89|5453740.68|
+--------------------+----+-----+---+----+------+--------------------+-------------------+---------+----------+
only showing top 3 rows



In [17]:
vancouver_df.fillna(0, subset=['HOUR', 'MINUTE']).show(3)

+--------------------+----+-----+---+----+------+--------------------+-------------------+---------+----------+
|                TYPE|YEAR|MONTH|DAY|HOUR|MINUTE|       HUNDRED_BLOCK|      NEIGHBOURHOOD|        X|         Y|
+--------------------+----+-----+---+----+------+--------------------+-------------------+---------+----------+
|Offence Against a...|2003|    5| 17|   0|     0|OFFSET TO PROTECT...|               null|      0.0|       0.0|
|  Theft from Vehicle|2003|    1|  7|  10|     0|CARDERO ST / W GE...|           West End|490503.48|5459766.67|
|  Theft from Vehicle|2003|    7| 27|   5|     0|    53XX CHAMBERS ST|Renfrew-Collingwood|496145.89|5453740.68|
+--------------------+----+-----+---+----+------+--------------------+-------------------+---------+----------+
only showing top 3 rows



In [18]:
vancouver_df.fillna('Unknown', subset=['NEIGHBOURHOOD']).show(3)

+--------------------+----+-----+---+----+------+--------------------+-------------------+---------+----------+
|                TYPE|YEAR|MONTH|DAY|HOUR|MINUTE|       HUNDRED_BLOCK|      NEIGHBOURHOOD|        X|         Y|
+--------------------+----+-----+---+----+------+--------------------+-------------------+---------+----------+
|Offence Against a...|2003|    5| 17|null|  null|OFFSET TO PROTECT...|            Unknown|      0.0|       0.0|
|  Theft from Vehicle|2003|    1|  7|  10|     0|CARDERO ST / W GE...|           West End|490503.48|5459766.67|
|  Theft from Vehicle|2003|    7| 27|   5|     0|    53XX CHAMBERS ST|Renfrew-Collingwood|496145.89|5453740.68|
+--------------------+----+-----+---+----+------+--------------------+-------------------+---------+----------+
only showing top 3 rows

